In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
data=pd.read_excel('../Data/한국마사회_경주상세정보(제주_부경)_20230720.xlsx')

In [3]:
unicol_list = []
for col in data.columns:
    if data[1:][col].nunique()<=1:
        unicol_list.append(col)
drop_data = data.drop(unicol_list,axis=1) 

In [4]:
unicol_list[24:]

['BU_2F-G_TIME',
 'BU_3F-G_TIME',
 'BU_1F-G_TIME',
 'BU_S1F_ACC_TIME',
 'BU_G8F_ACC_TIME',
 'BU_G6F_ACC_TIME',
 'BU_G4F_ACC_TIME',
 'BU_G3F_ACC_TIME',
 'BU_G2F_ACC_TIME',
 'BU_G1F_ACC_TIME']

In [5]:
drop_data["WG"] = drop_data["WG_HR"].str.extract(r'(\d+)(?=\()')
drop_data["WG"][0]="마체중"

drop_data["WC"] = drop_data["WG_HR"].str.extract(r'\((.*?)\)')
drop_data["WC"][0] = "체중증감"

drop_data["WC"][1:] = pd.to_numeric(drop_data["WC"][1:])
drop_data["WG"][1:] = pd.to_numeric(drop_data["WG"][1:])

drop_data = drop_data.drop(["WG_HR"],axis=1)

In [8]:
#한글 이름을 열로 바꿔줌
drop_data.columns=drop_data.iloc[0]
drop_data=drop_data[1:]
drop_data

1,20230701,1,800,제6등급,흐림,포화 (18%),1,9,한백제왕,3101848,...,0.13586776859504132,0.11140495867768595,0.1549636803874092,0.12106537530266344,0.15224879440999903,0.12154315520125972,0.14953271028037382,0.09968847352024922,0.2459016393442623,0.12568306010928962
2,20230701,1,800,제6등급,흐림,포화 (18%),2,2,명의로운,3102087,...,0.093869,0.099154,0.064677,0.074627,0.096586,0.104722,0.043732,0.069971,0.048973,0.127962
3,20230701,1,800,제6등급,흐림,포화 (18%),3,3,번개의꿈,3101489,...,0.156444,0.132137,0.158986,0.124424,0.082014,0.101251,0.048682,0.099391,0.109116,0.104972
4,20230701,1,800,제6등급,흐림,포화 (18%),4,6,고차원,3103528,...,0.080102,0.090273,0.071979,0.092545,0.082014,0.101251,0.048682,0.099391,0.109116,0.104972
5,20230701,1,800,제6등급,흐림,포화 (18%),5,1,위너고,3102911,...,0.105547,0.10015,0.065693,0.072993,0.093358,0.099067,0.0475,0.0775,0.084081,0.098655
6,20230701,1,800,제6등급,흐림,포화 (18%),6,5,백프로,3102675,...,0.114352,0.096819,0.126126,0.096096,0.135672,0.114962,0.11215,0.096573,0.142268,0.107216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29588,20190111,9,900,한4등급,흐림,양호 (8%),6,8,백년친구,1017787,...,0.078571,0.089286,0.0,0.0,0.116165,0.117293,0.102362,0.091864,0.117873,0.122998
29589,20190111,9,900,한4등급,흐림,양호 (8%),7,3,미래제주,1017443,...,0.156444,0.132137,0.158986,0.124424,0.102273,0.107801,0.125,0.111111,0.08775,0.125109
29590,20190111,9,900,한4등급,흐림,양호 (8%),8,1,남단여제,1017686,...,0.086402,0.105361,0.0,0.0,0.094704,0.102217,0.117143,0.142857,0.0954,0.088586
29591,20190111,9,900,한4등급,흐림,양호 (8%),9,4,섬문화,1017765,...,0.110809,0.117121,0.094421,0.077253,0.078531,0.093028,0.091667,0.075,0.072055,0.095424


In [7]:
#1착, 2착 횟수가 높다고 좋은 것이 아니라 승률이 높아야 좋다고 생각하기에, 경주마, 기수, 조교수, 위탁 각각의 승률을 새로운 열에 추가해주었다.
drop_data['경주마통산1착승률']=drop_data['경주마통산1착횟수']/drop_data['경주마통산출전횟수'].replace(0,1)
drop_data['경주마통산2찬승률']=drop_data['경주마통산2착횟수']/drop_data['경주마통산출전횟수'].replace(0,1)
drop_data['경주마최근1년1착승률']=drop_data['경주마최근1년1착횟수']/drop_data['경주마최근1년출전횟수'].replace(0,1)
drop_data['경주마최근1년2착승률']=drop_data['경주마최근1년2착횟수']/drop_data['경주마최근1년출전횟수'].replace(0,1)

drop_data['기수통산1착승률']=drop_data['기수통산1착횟수']/drop_data['기수통산출전횟수'].replace(0,1)
drop_data['기수통산2착승률']=drop_data['기수통산2착횟수']/drop_data['기수통산출전횟수'].replace(0,1)
drop_data['기수최근1년간1착승률']=drop_data['기수최근1년간1착횟수']/drop_data['기수최근1년간출전횟수'].replace(0,1)
drop_data['기수최근1년간2착승률']=drop_data['기수최근1년간2착횟수']/drop_data['기수최근1년간출전횟수'].replace(0,1)

drop_data['조교사통산1착승률']=drop_data['조교사통산1착횟수']/drop_data['조교사통산출전횟수'].replace(0,1)
drop_data['조교사통산2착승률']=drop_data['조교사통산2착횟수']/drop_data['조교사통산출전횟수'].replace(0,1)
drop_data['조교사최근1년간1착승률']=drop_data['조교사최근1년간1착횟수']/drop_data['조교사최근1년간출전횟수'].replace(0,1)
drop_data['조교사최근1년간2착승률']=drop_data['조교사최근1년간2착횟수']/drop_data['조교사최근1년간출전횟수'].replace(0,1)

drop_data['조교사위탁경주마1위승률']=drop_data['조교사위탁경주마1위횟수']/drop_data['조교사위탁경주마출전횟수'].replace(0,1)
drop_data['조교사위탁경주마2위승률']=drop_data['조교사위탁경주마2위횟수']/drop_data['조교사위탁경주마출전횟수'].replace(0,1)


In [51]:
#주로상태(함수율) 데이터가 포화(18%)로 되어 있는 것을 포화, 18로 바꿔주었다.
drop_data[['주로상태','주로상태_함수율']]=drop_data['주로상태(함수율)'].str.extract(r'(.*?) \((\d+)%\)')
drop_data=drop_data.drop('주로상태(함수율)',axis=1)

In [58]:
drop_data[['혈통','등급']]=drop_data['경주등급'].str.extract(r'(\D)(.*)')

In [65]:
drop_data['혈통']=drop_data['혈통'].str.replace('제','제주').str.replace('한','한라')

In [67]:
drop_data=drop_data.drop('경주등급',axis=1)

In [73]:
drop_data.to_excel('경마데이터.xls',index=False)

C:\Users\wjswp\AppData\Local\Temp\ipykernel_16784\3188981914.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  drop_data.to_excel('경마데이터.xls',index=False)


In [15]:
#다시시작
horse_data=pd.read_excel('경마데이터.xls')
horse_data.columns
drop_feature=['마번','기수통산출전횟수','기수최근1년간출전횟수','날씨','경주마통산1착횟수','경주마통산2착횟수','경주마최근1년1착횟수','경주마최근1년2착횟수','기수명','기수번호','기수연령','기수통산1착횟수','기수통산2착횟수','기수최근1년간1착횟수','기수최근1년간2착횟수','조교사명','조교사번호','조교사통산1착횟수','조교사통산2착횟수','조교사최근1년간1착횟수','조교사최근1년간2착횟수','조교사위탁경주마1위횟수','조교사위탁경주마2위횟수','주로상태']
horse_data=horse_data.drop(columns=drop_feature)

In [20]:
np.unique(horse_data['경주거리'])
horse_data_800=horse_data[horse_data['경주거리']==800]
horse_data_900=horse_data[horse_data['경주거리']==900]
horse_data_1000=horse_data[horse_data['경주거리']==1000]
horse_data_1100=horse_data[horse_data['경주거리']==1100]
horse_data_1200=horse_data[horse_data['경주거리']==1200]
horse_data_1400=horse_data[horse_data['경주거리']==1400]
horse_data_1610=horse_data[horse_data['경주거리']==1610]
horse_data_1700=horse_data[horse_data['경주거리']==1700]
horse_data_1800=horse_data[horse_data['경주거리']==1800]

In [ ]:
pd.set_option('display.max_columns',None)
horse_data_800.columns
#서울제주 1코너 2코너
drop_feature_800=['서울,제주1코너구간통과순위(서울,제주공통)','서울,제주2코너구간통과순위(서울,제주공통)','제주1코너통과기록','제주2코너통과기록']
horse_data_800=horse_data_800.drop(columns=drop_feature_800)

In [31]:
majong_0=pd.read_excel('0~1000마종데이터.xls')
majong_1=pd.read_excel('1001~1500마종데이터.xls')
majong_2=pd.read_excel('1501~마종데이터.xls')
print(majong_1)


        ini_price                                              sicks  \
0            자가생산  ['좌전지파행 2023/02/26', '양전지파행 2023/02/25', '좌전지파...   
1    35,000천원(개별)  ['정치 2023/05/08', '낭치 2023/05/08', '감기 2020/04...   
2    10,000천원(개별)  ['임신검사 2020/03/18', '임신검사 2019/11/21', '정치 201...   
3    35,000천원(개별)  ['좌전지파행 2020/12/06', '좌전지파행 2020/12/05', '좌전지파...   
4    10,000천원(개별)                                                 []   
..            ...                                                ...   
494           NaN  ['감기 2021/08/06', '감기 2021/08/05', '감기 2021/08...   
495           NaN                                                 []   
496  36,997천원(개별)  ['좌전지파행 2022/11/13', '좌전지파행 2022/10/15', '우전지파...   
497  29,000천원(개별)              ['우전지파행 2021/09/12', '정치 2021/08/05']   
498          자가생산  ['양전지파행 2023/07/27', '정치 2021/07/08', '낭치 2021...   

                                                 ranks  \
0    ['제3 2023/01/08 ~ 현재', '제4 2021/07/04 ~ 2023/0...   
1    ['제4 2023/08/1

In [72]:
#len(pd.concat([majong_0,majong_1,majong_2]))
horse_list=pd.read_csv("../Data/horse_list.csv")
#len(horse_list)
#majong_0 1000개
#majong_1 499개
#2001개 1개는 majong_1 1번 돌풍신화 index=1000
#1개는 majong_2 1번 황룡시대 추가 index=1500
horse_list.drop([1000,1500],inplace=True)
horse_list=horse_list['0']


In [148]:
majong_data=pd.concat([majong_0,majong_1,majong_2])
majong_data['name']=horse_list

In [150]:
majong_feature=['ranks','horse_rate','dad_record','dad_rate','mom_record','mom_rate','name']
majong_data=majong_data[majong_feature]

In [151]:
majong_data.reset_index(drop=True,inplace=True)
dad_rate=np.zeros(len(majong_data))
mom_rate=np.zeros(len(majong_data))
#아빠 승률
for i in range(len(majong_data)):
    dad=majong_data['dad_record'][i]
    if (dad=='None' or dad=='전 (////)' or dad=='0전 (0/0/0/0/0)'):
        dad_rate[i]=None
    else:
        b=re.findall(r'\d+',dad)
        b=list(map(int,b))
        dad_rate[i]=round((b[1]+b[2]+b[3])/b[0],2)


#엄마 승률
for i in range(len(majong_data)):
    mom=majong_data['mom_record'][i]
    if (mom=='None' or mom=='전 (////)' or mom=='0전 (0/0/0/0/0)'):
        mom_rate[i]=None
    else:
        b=re.findall(r'\d+',mom)
        b=list(map(int,b))
        mom_rate[i]=round((b[1]+b[2]+b[3])/b[0],2)

In [154]:
majong_data['mom_rating']=mom_rate
majong_data['dad_rating']=dad_rate

C:\Users\wjswp\AppData\Local\Temp\ipykernel_7784\1244415613.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  majong_data['mom_rating']=mom_rate
C:\Users\wjswp\AppData\Local\Temp\ipykernel_7784\1244415613.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  majong_data['dad_rating']=dad_rate


In [155]:
majong_data

,ranks,horse_rate,dad_record,dad_rate,mom_record,mom_rate,name,mom_rating,dad_rating
0,"['제4 2020/01/01 ~ 2021/05/12', '제3 2019/08/04 ...","['2020/12/19-5R 59', '2020/11/21-6R 59', '2020...",42전 (6/8/5/7/5),단승 : 14.3% 복승 : 33.3% 연승 : 45.2%,0전 (0/0/0/0/0),단승 : 0.0% 복승 : 0.0% 연승 : 0.0%,무대바람,NaN,0.45
1,"['제5 2020/01/01 ~ 2020/05/07', '제4 2018/09/02 ...","['2020/02/22-3R 40', '2020/01/31-2R 40', '2020...",2전 (2/0/0/0/0),단승 : 100.0% 복승 : 100.0% 연승 : 100.0%,None,None,마의질주,NaN,1.00
2,"['제3 2020/08/23 ~ 2021/01/27', '제4 2020/01/01 ...","['2021/01/22-8R 61', '2020/12/20-7R 61', '2020...",42전 (6/8/5/7/5),단승 : 14.3% 복승 : 33.3% 연승 : 45.2%,18전 (5/4/2/2/1),단승 : 27.8% 복승 : 50.0% 연승 : 61.1%,쾌속비행,0.61,0.45
3,"['제4 2018/08/26 ~ 2020/07/22', '제1 2016/08/19 ...","['2019/12/19-3R 44', '2019/07/13-2R 45', '2019...",None,None,None,None,포르티우스,NaN,NaN
4,"['제4 2022/01/01 ~ 2023/04/20', '제3 2021/09/12 ...","['2023/04/15-4R 52', '2023/03/16-2R 52', '2022...",None,None,33전 (1/1/1/5/2),단승 : 3.0% 복승 : 6.1% 연승 : 9.1%,광명일호,0.09,NaN
...,...,...,...,...,...,...,...,...,...
1996,"['제5 2023/06/25 ~ 현재', '제6 2023/06/08 ~ 2023/0...","['2023/07/28-4R 19', '2023/07/08-2R 20', '2023...",78전 (7/5/10/7/13),단승 : 9.0% 복승 : 15.4% 연승 : 28.2%,21전 (7/4/2/1/0),단승 : 33.3% 복승 : 52.4% 연승 : 61.9%,우주소녀,0.62,0.28
1997,"['제6 2023/06/08 ~ 2023/07/19', '미 2022/11/02 ~...","['2023/07/15-1R ', '2023/06/24-1R ', '2023/06/...",36전 (6/2/0/0/1),단승 : 16.7% 복승 : 22.2% 연승 : 22.2%,37전 (2/5/3/4/6),단승 : 5.4% 복승 : 18.9% 연승 : 27.0%,상가으뜸,0.27,0.22
1998,"['제5 2023/07/02 ~ 현재', '제6 2023/06/22 ~ 2023/0...","['2023/09/22-2R 37', '2023/08/26-2R 28', '2023...",23전 (4/3/2/1/2),단승 : 17.4% 복승 : 30.4% 연승 : 39.1%,26전 (9/5/3/1/1),단승 : 34.6% 복승 : 53.8% 연승 : 65.4%,고색창연,0.65,0.39
1999,"['C4 2012/01/28 ~ 2012/04/04', '신 2012/01/12 ~...","['2012/03/10-7R ', '2012/02/10-8R ', '2012/01/...",None,None,None,None,광산질주,NaN,NaN
